# 2024 Albums of the Year

Welcome, casually observer, welcome to my year long journey to rank the 100 best albums of the year. Over the past 10.5 months, I have been listening to about a dozen albums a week, and putting the best albums in a spotify album called 2024. 

I've also been scrobbling every song I listen to on spotify to my account on last.fm. 

My intention is to pull the data from my 2024 playlist, my Liked Songs playlist, and my last.fm scrobbles, to pull data about each album, rank by scrobbles, likes, and a few feature engineering to spice things up. 

In [1]:
import requests
import pandas as pd
import spotipy
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from datetime import datetime
import os
import sqlite3

In [ ]:
# Define the path to the Data folder
data_folder = os.path.join(os.getcwd(), 'Data')

# Create Data folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

API_KEY = '74a510ecc9fc62bf3e0edc6adc2e99f9'
USERNAME = 'Strusz_Music'
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

# Get today's date in YYYY-MM-DD format
today = datetime.today().strftime('%Y-%m-%d')

# Set up SQLite database with today's date in the filename, saved inside the Data folder
db_filename = os.path.join(data_folder, f'lastfm_scrobbles_{today}.db')
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# Create table if it doesn't exist (removed play_count, duration, and favorite)
cursor.execute('''
CREATE TABLE IF NOT EXISTS scrobbles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    artist TEXT,
    title TEXT,
    album TEXT,
    release_date TEXT,
    album_art TEXT,
    timestamp INTEGER
)
''')

# Define the Unix timestamp for January 1, 2024 (00:00:00)
start_of_2024 = int(datetime(2024, 1, 1).timestamp())

# Function to get scrobbles
def get_scrobbles(page=1):
    params = {
        'method': 'user.getrecenttracks',
        'user': USERNAME,
        'api_key': API_KEY,
        'format': 'json',
        'limit': 200,
        'page': page
    }
    response = requests.get(BASE_URL, params=params)
    return response.json()

# Function to insert scrobbles into the database (removed play_count, duration, and favorite)
def insert_scrobbles(scrobbles):
    for track in scrobbles:
        artist = track.get('artist', {}).get('#text', 'Unknown')
        title = track.get('name', 'Unknown')
        album = track.get('album', {}).get('#text', 'Unknown')
        release_date = track.get('album', {}).get('releasedate', 'Unknown')
        album_art = track.get('image', [{}])[3].get('#text', '')  # Get the medium-sized image URL
        timestamp_str = track.get('date', {}).get('uts', '0')
        
        # Convert the timestamp from string to integer
        timestamp = int(timestamp_str)

        # Only insert scrobbles from 2024 onwards
        if timestamp >= start_of_2024:
            cursor.execute('''
            INSERT INTO scrobbles (artist, title, album, release_date, album_art, timestamp) 
            VALUES (?, ?, ?, ?, ?, ?)
            ''', (artist, title, album, release_date, album_art, timestamp))
    
    conn.commit()

# Fetch and store scrobbles
def fetch_and_store_scrobbles():
    page = 1
    while True:
        data = get_scrobbles(page)
        tracks = data.get('recenttracks', {}).get('track', [])
        if not tracks:
            break
        insert_scrobbles(tracks)
        page += 1

# Run the process
fetch_and_store_scrobbles()

# Close the database connection
conn.close()

I used DBbrowser to convert the db file. 

## IMPORT THE LAST.FM SCROBBLES.CSV

In [13]:
scrobbles = pd.read_csv('Data/scrobbles2.csv')

In [15]:
scrobbles.head()

,id,artist,title,album,release_date,album_art,timestamp
0,1,Kendrick Lamar,"gnx (feat. hitta j3, youngthreat, peysoh)",GNX,Unknown,https://lastfm.freetls.fastly.net/i/u/300x300/...,1732664212
1,2,Kendrick Lamar,heart pt. 6,GNX,Unknown,https://lastfm.freetls.fastly.net/i/u/300x300/...,1732663923
2,3,Kendrick Lamar,heart pt. 6,GNX,Unknown,https://lastfm.freetls.fastly.net/i/u/300x300/...,1732660866
3,4,Kendrick Lamar,peekaboo (feat. AzChike),GNX,Unknown,https://lastfm.freetls.fastly.net/i/u/300x300/...,1732660710
4,5,Kendrick Lamar,"dodger blue (feat. wallie the sensei, siete7x,...",GNX,Unknown,https://lastfm.freetls.fastly.net/i/u/300x300/...,1732660579


In [17]:
scrobbles.columns

Index(['id', 'artist', 'title', 'album', 'release_date', 'album_art',
       'timestamp'],
      dtype='object')

In [19]:
# Drop the columns and update the DataFrame
scrobbles.drop(['id', 'release_date'], axis=1, inplace=True)

# Confirm the columns have been dropped
scrobbles.columns

Index(['artist', 'title', 'album', 'album_art', 'timestamp'], dtype='object')

Checking for double scrobbles, last.fm once and a while will count a song twice that you only listened to once. 

In [21]:
scrobbles[scrobbles.duplicated(subset=['artist', 'title', 'album', 'timestamp'], keep=False)]

,artist,title,album,album_art,timestamp
14915,Ivor Cutler Trio,Cockadoodledon't,Ludo,https://lastfm.freetls.fastly.net/i/u/300x300/...,1708646297
14916,Ivor Cutler Trio,Cockadoodledon't,Ludo,https://lastfm.freetls.fastly.net/i/u/300x300/...,1708646297


Dropping one copy of each of these double scrobbles.

In [23]:
scrobbles.drop_duplicates(subset=['artist', 'title', 'album', 'timestamp'], inplace=True)

---

## Checking for Null Values:

In [25]:
scrobbles.isnull().sum()

artist        0
title         0
album        80
album_art    80
timestamp     0
dtype: int64

In [27]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', 80)  # Show up to 80 rows
pd.set_option('display.max_columns', None)  # Show all columns

scrobbles[scrobbles['album'].isnull() | scrobbles['album_art'].isnull()]

,artist,title,album,album_art,timestamp
9285,889 Radio Milwaukee,Discover New Music,NaN,NaN,1718628917
9288,889 Radio Milwaukee,Discover New Music,NaN,NaN,1718628469
9289,Jon Batiste,Uneasy,NaN,NaN,1718628157
11852,889 Radio Milwaukee,Discover New Music,NaN,NaN,1713143094
11853,889 Radio Milwaukee,Discover New Music,NaN,NaN,1713142876
11854,Tiawa,Sonhos Cor de Rosa,NaN,NaN,1713142722
12123,889 Radio Milwaukee,Discover New Music,NaN,NaN,1712669006
12124,Norah Jones,Running,NaN,NaN,1712668878
12592,Nation of Language,"Too Much, Enough",NaN,NaN,1711653831
12593,Justice,One Night/All Night (Ft. Tame Impala),NaN,NaN,1711653406


These are radio scrobbles from a radio app, mostly. I'm comfortable removing them. 

In [29]:
# Drop rows with null values in any column
scrobbles.dropna(inplace=True)

---

## Counting the Scrobbles

In [31]:
# Extract the primary artist (first one listed) before grouping
scrobbles['artist'] = scrobbles['artist'].str.split(',').str[0].str.strip()

# Group by the cleaned artist and album to count the number of times each album was played
album_counts = scrobbles.groupby(['artist', 'album', 'album_art']).size().reset_index(name='scrobb_num')

In [33]:
# Export the result to CSV
output_path = 'Data/scrobbles_summary.csv'
album_counts.to_csv(output_path, index=False)

In [35]:
album_counts.to_csv('Data/scrobbles_summary.csv', index=False)

In [106]:
album_counts.columns

Index(['artist', 'album', 'album_art', 'scrobb_num'], dtype='object')

> Previewing the data, albums that have a featured artist for a song or more were counted as different albums. I noticed a few where the artist name was seperated by a comma with the featured artist.
>
> i took care of it in the above cell blocks, but want to leave this here to maybe incorporate into the blog narrative. 

# IMPORTING LIKED.CSV
These are all of the songs I've ever liked on spotify, downloaded as a csv from exportify.net

In [38]:
liked = pd.read_csv('Data/liked.csv', on_bad_lines='skip')

In [40]:
liked

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,Record Label,\t\t\t\tDanceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,4asjU3sDnn3002bTJIg2vZ,Missed the Boat,We Were Dead Before The Ship Even Sank,Modest Mouse,2007-03-20,264813,54,NaN,2024-11-13T17:20:04Z,"alternative rock,chamber pop,indie rock,indiet...",Epic,0.710,0.698,2,-5.797,1,0.0332,0.069700,0.000000,0.4780,0.8520,104.007,4
1,0Fe3WxeO6lZZxj7ytvbDUh,Dashboard,We Were Dead Before The Ship Even Sank,Modest Mouse,2007-03-20,246173,58,NaN,2024-11-13T16:06:37Z,"alternative rock,chamber pop,indie rock,indiet...",Epic,0.639,0.903,11,-5.013,0,0.0371,0.009630,0.000071,0.1740,0.9600,135.938,4
2,45tBkNCWL867mdQDjSmQ6q,Too Many Things,No Obligation,The Linda Lindas,2024-10-11,155893,36,NaN,2024-11-11T23:15:57Z,riot grrrl,Epitaph,0.572,0.900,7,-2.570,1,0.0436,0.000215,0.267000,0.4280,0.8620,156.938,4
3,7uHpXA3rYfs1EqFHogCwDj,All In My Head,No Obligation,The Linda Lindas,2024-10-11,201187,47,NaN,2024-11-11T23:08:47Z,riot grrrl,Epitaph,0.536,0.861,0,-4.443,1,0.0511,0.011100,0.000004,0.1140,0.7130,145.017,4
4,6vyKPAESyy0nmIiOCWwyLO,Jerry's Song,Seed of a Seed,Haley Heynderickx,2024-11-01,195360,45,NaN,2024-11-09T18:09:28Z,"countrygaze,indie pop,portland indie,small room",Mama Bird Recording Co.,0.405,0.234,8,-15.601,0,0.0338,0.847000,0.089700,0.1090,0.1990,119.977,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5502,1dnTzkUEdwm9jrmdpKSfWU,Mercy Mercy Me (The Ecology),What's Going On,Marvin Gaye,1971-05-21,196466,0,NaN,2020-12-20T19:19:33Z,"classic soul,motown,neo soul,northern soul,qui...",Motown,0.499,0.633,6,-12.145,0,0.0445,0.373000,0.716000,0.0876,0.4740,93.238,4
5503,4WPaMPLhdPJyZvFSHTzHOk,Angels,2045,Parris Chariz,2020-07-17,186184,11,NaN,2020-12-20T19:19:33Z,"christian hip hop,christian trap",RMG Amplify / World 45,0.609,0.700,0,-7.018,1,0.0407,0.002670,0.000000,0.3650,0.1560,169.987,4
5504,0kkKlkEIHdcQbTQlEkAdgp,Snails,Dog Problems (Bonus Track Version),The Format,2006-07-11,249133,0,NaN,2020-12-20T19:19:33Z,emo,The Vanity Label,0.682,0.548,4,-8.825,1,0.0240,0.095200,0.060200,0.2730,0.6190,93.998,4
5505,0Mfwz6JTTMrwXo2XIj7WGU,Salina,Emotionalism (Bonus Track Version),The Avett Brothers,2007-05-15,284040,30,NaN,2020-12-20T19:19:32Z,"indie folk,new americana,north carolina indie,...",Ramseur Records,0.452,0.235,1,-10.178,1,0.0435,0.847000,0.000438,0.1350,0.0654,118.097,4


In [42]:
liked.columns = liked.columns.str.strip().str.replace('\t', '', regex=True)

Removing likes from before 2024:

In [44]:
# Use .loc explicitly to avoid the warning
liked = liked.loc[liked['Added At'] >= '2024-01-01'].copy()

# Convert 'Added At' to datetime format
liked['Added At'] = pd.to_datetime(liked['Added At'])

# Confirm the filter worked
liked.head()

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,Record Label,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,4asjU3sDnn3002bTJIg2vZ,Missed the Boat,We Were Dead Before The Ship Even Sank,Modest Mouse,2007-03-20,264813,54,NaN,2024-11-13 17:20:04+00:00,"alternative rock,chamber pop,indie rock,indiet...",Epic,0.710,0.698,2,-5.797,1,0.0332,0.069700,0.000000,0.478,0.852,104.007,4
1,0Fe3WxeO6lZZxj7ytvbDUh,Dashboard,We Were Dead Before The Ship Even Sank,Modest Mouse,2007-03-20,246173,58,NaN,2024-11-13 16:06:37+00:00,"alternative rock,chamber pop,indie rock,indiet...",Epic,0.639,0.903,11,-5.013,0,0.0371,0.009630,0.000071,0.174,0.960,135.938,4
2,45tBkNCWL867mdQDjSmQ6q,Too Many Things,No Obligation,The Linda Lindas,2024-10-11,155893,36,NaN,2024-11-11 23:15:57+00:00,riot grrrl,Epitaph,0.572,0.900,7,-2.570,1,0.0436,0.000215,0.267000,0.428,0.862,156.938,4
3,7uHpXA3rYfs1EqFHogCwDj,All In My Head,No Obligation,The Linda Lindas,2024-10-11,201187,47,NaN,2024-11-11 23:08:47+00:00,riot grrrl,Epitaph,0.536,0.861,0,-4.443,1,0.0511,0.011100,0.000004,0.114,0.713,145.017,4
4,6vyKPAESyy0nmIiOCWwyLO,Jerry's Song,Seed of a Seed,Haley Heynderickx,2024-11-01,195360,45,NaN,2024-11-09 18:09:28+00:00,"countrygaze,indie pop,portland indie,small room",Mama Bird Recording Co.,0.405,0.234,8,-15.601,0,0.0338,0.847000,0.089700,0.109,0.199,119.977,4


In [46]:
liked.shape

(314, 23)

In [51]:
liked.columns

Index(['Track ID', 'Track Name', 'Album Name', 'Artist Name(s)',
       'Release Date', 'Duration (ms)', 'Popularity', 'Added By', 'Added At',
       'Genres', 'Record Label', 'Danceability', 'Energy', 'Key', 'Loudness',
       'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo', 'Time Signature'],
      dtype='object')

## IMPORTING 2024.csv

In [53]:
best = pd.read_csv('Data/2024.csv', on_bad_lines='skip')


In [55]:
best.head()

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,Record Label,\t\t\t\tDanceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,0UOeq7bSskoJa4cJaJOmFS,Ticking,Letter to Self,SPRINTS,2024-01-05,186949,32,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.376,0.620,11.0,-6.490,1.0,0.3440,0.02500,0.076500,0.0934,0.291,175.574,4.0
1,02bA26OEe0nNFyE3YcNx4K,Heavy,Letter to Self,SPRINTS,2024-01-05,207409,45,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.348,0.705,11.0,-5.925,1.0,0.0591,0.00435,0.000738,0.0877,0.189,88.581,4.0
2,7IPDhCIQlpvxVxtC1Q7Jq4,Cathedral,Letter to Self,SPRINTS,2024-01-05,179694,31,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.535,0.699,7.0,-6.231,1.0,0.0473,0.00978,0.002700,0.0887,0.397,119.056,4.0
3,65fPteG9ctHt2rrJxlbMr8,Shaking Their Hands,Letter to Self,SPRINTS,2024-01-05,222489,30,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.214,0.851,4.0,-5.658,0.0,0.0533,0.19900,0.108000,0.1330,0.551,89.485,4.0
4,4UgkFdXpJD0fhw06BMk0bz,Adore Adore Adore,Letter to Self,SPRINTS,2024-01-05,157766,37,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.412,0.877,4.0,-4.401,0.0,0.2570,0.01070,0.000107,0.1010,0.402,176.054,4.0


In [57]:
best.columns = best.columns.str.strip().str.replace('\t', '')
best.columns

Index(['Track ID', 'Track Name', 'Album Name', 'Artist Name(s)',
       'Release Date', 'Duration (ms)', 'Popularity', 'Added By', 'Added At',
       'Genres', 'Record Label', 'Danceability', 'Energy', 'Key', 'Loudness',
       'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo', 'Time Signature'],
      dtype='object')

## Cleaning Artist Names and Album Names

In [65]:
# Ensure that artist names have no leading/trailing spaces and commas are handled properly
liked['Artist Name(s)'] = liked['Artist Name(s)'].str.replace(r',\s*', ',', regex=True).str.strip()
best['Artist Name(s)'] = best['Artist Name(s)'].str.replace(r',\s*', ',', regex=True).str.strip()

# Clean the album names too (if needed, removing any leading/trailing spaces)
liked['Album Name'] = liked['Album Name'].str.strip()
best['Album Name'] = best['Album Name'].str.strip()

In [67]:
# Keep only the first artist before the comma
liked['Artist Name(s)'] = liked['Artist Name(s)'].str.split(',').str[0].str.strip()
best['Artist Name(s)'] = best['Artist Name(s)'].str.split(',').str[0].str.strip()


In [69]:
liked.head(5)

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,Record Label,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,4asjU3sDnn3002bTJIg2vZ,Missed the Boat,We Were Dead Before The Ship Even Sank,Modest Mouse,2007-03-20,264813,54,NaN,2024-11-13 17:20:04+00:00,"alternative rock,chamber pop,indie rock,indiet...",Epic,0.710,0.698,2,-5.797,1,0.0332,0.069700,0.000000,0.478,0.852,104.007,4
1,0Fe3WxeO6lZZxj7ytvbDUh,Dashboard,We Were Dead Before The Ship Even Sank,Modest Mouse,2007-03-20,246173,58,NaN,2024-11-13 16:06:37+00:00,"alternative rock,chamber pop,indie rock,indiet...",Epic,0.639,0.903,11,-5.013,0,0.0371,0.009630,0.000071,0.174,0.960,135.938,4
2,45tBkNCWL867mdQDjSmQ6q,Too Many Things,No Obligation,The Linda Lindas,2024-10-11,155893,36,NaN,2024-11-11 23:15:57+00:00,riot grrrl,Epitaph,0.572,0.900,7,-2.570,1,0.0436,0.000215,0.267000,0.428,0.862,156.938,4
3,7uHpXA3rYfs1EqFHogCwDj,All In My Head,No Obligation,The Linda Lindas,2024-10-11,201187,47,NaN,2024-11-11 23:08:47+00:00,riot grrrl,Epitaph,0.536,0.861,0,-4.443,1,0.0511,0.011100,0.000004,0.114,0.713,145.017,4
4,6vyKPAESyy0nmIiOCWwyLO,Jerry's Song,Seed of a Seed,Haley Heynderickx,2024-11-01,195360,45,NaN,2024-11-09 18:09:28+00:00,"countrygaze,indie pop,portland indie,small room",Mama Bird Recording Co.,0.405,0.234,8,-15.601,0,0.0338,0.847000,0.089700,0.109,0.199,119.977,4


In [61]:
best.head(5)

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,Record Label,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,0UOeq7bSskoJa4cJaJOmFS,Ticking,Letter to Self,SPRINTS,2024-01-05,186949,32,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.376,0.620,11.0,-6.490,1.0,0.3440,0.02500,0.076500,0.0934,0.291,175.574,4.0
1,02bA26OEe0nNFyE3YcNx4K,Heavy,Letter to Self,SPRINTS,2024-01-05,207409,45,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.348,0.705,11.0,-5.925,1.0,0.0591,0.00435,0.000738,0.0877,0.189,88.581,4.0
2,7IPDhCIQlpvxVxtC1Q7Jq4,Cathedral,Letter to Self,SPRINTS,2024-01-05,179694,31,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.535,0.699,7.0,-6.231,1.0,0.0473,0.00978,0.002700,0.0887,0.397,119.056,4.0
3,65fPteG9ctHt2rrJxlbMr8,Shaking Their Hands,Letter to Self,SPRINTS,2024-01-05,222489,30,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.214,0.851,4.0,-5.658,0.0,0.0533,0.19900,0.108000,0.1330,0.551,89.485,4.0
4,4UgkFdXpJD0fhw06BMk0bz,Adore Adore Adore,Letter to Self,SPRINTS,2024-01-05,157766,37,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.412,0.877,4.0,-4.401,0.0,0.2570,0.01070,0.000107,0.1010,0.402,176.054,4.0


## Counting the number of Tracks Per Album

In [71]:
# Count the number of tracks for each album
album_track_counts = best.groupby('Album Name').size().reset_index(name='track_count')

# Merge the track counts into the best DataFrame by matching on the 'Album Name'
best = best.merge(album_track_counts, on='Album Name', how='left')

# Now, 'best' will have a 'track_count' column with the number of tracks for each album
best.head()

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,Record Label,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,track_count
0,0UOeq7bSskoJa4cJaJOmFS,Ticking,Letter to Self,SPRINTS,2024-01-05,186949,32,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.376,0.620,11.0,-6.490,1.0,0.3440,0.02500,0.076500,0.0934,0.291,175.574,4.0,11
1,02bA26OEe0nNFyE3YcNx4K,Heavy,Letter to Self,SPRINTS,2024-01-05,207409,45,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.348,0.705,11.0,-5.925,1.0,0.0591,0.00435,0.000738,0.0877,0.189,88.581,4.0,11
2,7IPDhCIQlpvxVxtC1Q7Jq4,Cathedral,Letter to Self,SPRINTS,2024-01-05,179694,31,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.535,0.699,7.0,-6.231,1.0,0.0473,0.00978,0.002700,0.0887,0.397,119.056,4.0,11
3,65fPteG9ctHt2rrJxlbMr8,Shaking Their Hands,Letter to Self,SPRINTS,2024-01-05,222489,30,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.214,0.851,4.0,-5.658,0.0,0.0533,0.19900,0.108000,0.1330,0.551,89.485,4.0,11
4,4UgkFdXpJD0fhw06BMk0bz,Adore Adore Adore,Letter to Self,SPRINTS,2024-01-05,157766,37,mmr4r23xnc6oh1c77lysfbqg4,2024-01-05T12:56:38Z,"crank wave,irish indie rock,irish post-punk",City Slang,0.412,0.877,4.0,-4.401,0.0,0.2570,0.01070,0.000107,0.1010,0.402,176.054,4.0,11


In [73]:
# Export the cleaned 'best' DataFrame to a CSV file
best.to_csv('Data/2024_cleaned.csv', index=False)


## Finding the Liked Songs from 2024's Best Albums

In [100]:
# Merge liked with best DataFrame on 'Album Name' and 'Artist Name(s)', including 'track_count'
liked_from_best = liked.merge(
    best[['Album Name', 'Artist Name(s)', 'Track Name', 'Popularity', 'track_count']], 
    on=['Album Name', 'Artist Name(s)'], 
    how='inner'
)

# Rename columns to remove suffixes and keep the ones you want
liked_from_best = liked_from_best.rename(columns={
    'Track Name_x': 'Track Name',
    'Popularity_x': 'Popularity'
})

# Drop unnecessary columns (the ones with '_y' suffix)
liked_from_best = liked_from_best.drop(columns=['Track Name_y', 'Popularity_y'])

# Select only the relevant columns after the merge, including 'track_count'
liked_from_best = liked_from_best[['Track Name', 'Album Name', 'Artist Name(s)', 
                                   'Release Date', 'Popularity', 'Genres', 'Record Label', 'track_count']]

# Group by 'Album Name' and 'Artist Name(s)', and for each group, find the song with the highest popularity
most_popular_liked = liked_from_best.loc[liked_from_best.groupby(['Album Name', 'Artist Name(s)'])['Popularity'].idxmax()]

# Now for albums with no liked songs, we select the most popular song from the 'best' DataFrame, including 'track_count'
most_popular_all = best[['Album Name', 'Artist Name(s)', 'Track Name', 'Popularity', 
                         'Release Date', 'Genres', 'Record Label', 'track_count']].sort_values(
    'Popularity', ascending=False
).drop_duplicates('Album Name')

# Merge both datasets: liked songs and the most popular for albums without liked songs
final_data = pd.concat([most_popular_liked, most_popular_all]).drop_duplicates(
    subset=['Album Name', 'Artist Name(s)'], keep='first'
)

# Sort by album to make it easier to read
final_data = final_data.sort_values(['Album Name', 'Artist Name(s)'])

# Display the final result, including 'track_count'
final_data = final_data[['Album Name', 'Artist Name(s)', 'Track Name', 
                         'Popularity', 'Release Date', 'Genres', 'Record Label', 'track_count']]


In [102]:
final_data

,Album Name,Artist Name(s),Track Name,Popularity,Release Date,Genres,Record Label,track_count
620,$10 Cowboy,Charley Crockett,$10 Cowboy,50,2024-04-26,"new americana,southern americana",Son of Davy,12
78,4 Hits & A Miss - The Essential Richard Swift,Richard Swift,Dirty Jim,23,2024-11-01,NaN,Secretly Canadian,14
1678,A Minute...,Myles Smith,Nice To Meet You,79,2024-11-08,singer-songwriter pop,RCA Records Label,7
1446,A Modern Day Distraction,Jake Bugg,Never Said Goodbye,47,2024-10-04,"modern rock,nottingham indie",RCA Records Label,12
1151,A Sacred Way of Living,Easy Sleeper,Timekeeper,25,2024-08-30,NaN,Easy Sleeper,9
...,...,...,...,...,...,...,...,...
1160,Willson,Ashe,Running Out Of Time,47,2024-09-06,alt z,Ashe,12
1121,Wish On The Bone,Why Bonnie,Wish On The Bone,26,2024-08-30,"bedroom pop,bubblegrunge,dreamgaze",Fire Talk,11
633,Worldviews,Annabel,We Are Where We Are,2,2024-06-14,midwest emo,Tiny Engines,12
862,another year,Brother Bird,state of mind,18,2024-03-08,NaN,Easy Does It Records,11


In [90]:
print(final_data.columns)


Index(['Album Name', 'Artist Name(s)', 'Track Name', 'Popularity',
       'Release Date', 'Genres', 'Record Label', 'track_count'],
      dtype='object')


Saving this list that essentially has one row for each album, along with other important metrics for my final product. 

In [92]:
final_data.to_csv('Data/best_song.csv', index=False)

---

In [ ]:
album_counts

In [ ]:
final_data

In [114]:
print("final_data columns:", final_data.columns)
print("track_counts columns:", track_counts.columns)
print("album_counts columns:", album_counts.columns)


final_data columns: Index(['Album Name', 'Artist Name(s)', 'Track Name', 'Popularity',
       'Release Date', 'Genres', 'Record Label', 'track_count',
       'num_of_tracks'],
      dtype='object')
track_counts columns: Index(['Album Name', 'Artist Name(s)', 'num_of_tracks'], dtype='object')
album_counts columns: Index(['artist', 'album', 'album_art', 'scrobb_num'], dtype='object')


In [110]:
# Count the number of tracks per album and artist
track_counts = best.groupby(['Album Name', 'Artist Name(s)']).size().reset_index(name='num_of_tracks')


In [118]:
# Calculate the number of liked songs per album and artist
liked_count = liked.groupby(['Album Name', 'Artist Name(s)']).size().reset_index(name='liked_count')

# Merge liked_count with final_data
final_data = pd.merge(final_data, liked_count, on=['Album Name', 'Artist Name(s)'], how='left')

# Merge track counts with final_data
final_data = pd.merge(final_data, track_counts, on=['Album Name', 'Artist Name(s)'], how='left')

# Merge album_art and scrobb_num with final_data from album_counts
final_data = pd.merge(final_data, album_counts[['Album Name', 'Artist Name(s)', 'album_art', 'scrobb_num']], 
                      on=['Album Name', 'Artist Name(s)'], how='left')

# Check for '_x' and '_y' columns and resolve duplicates for 'num_of_tracks'
if 'num_of_tracks_x' in final_data.columns:
    final_data['num_of_tracks'] = final_data['num_of_tracks_x']
elif 'num_of_tracks_y' in final_data.columns:
    final_data['num_of_tracks'] = final_data['num_of_tracks_y']

# Drop unnecessary columns if they exist
final_data = final_data.drop(columns=['num_of_tracks_x', 'num_of_tracks_y'], errors='ignore')

# Reorder the columns for clarity
final_data = final_data[['Album Name', 'Artist Name(s)', 'Track Name', 
                         'Popularity', 'Release Date', 'Genres', 'Record Label', 
                         'track_count', 'liked_count', 'num_of_tracks', 
                         'album_art', 'scrobb_num']]

# Display a sample of the data to confirm it worked
final_data.head()


KeyError: "['album_art', 'scrobb_num'] not in index"

In [ ]:
# Check the columns in final_data after merge
print("Columns in final_data after merge:")
print(final_data.columns)

In [ ]:
# Inspect the columns in final_data after the merge to ensure the right column is there
print(final_data.columns)

# If 'num_of_tracks_y' exists, assign it to 'num_of_tracks'
if 'num_of_tracks_y' in final_data.columns:
    final_data['num_of_tracks'] = final_data['num_of_tracks_y']
    # Drop the unnecessary columns 'num_of_tracks_x' and 'num_of_tracks_y'
    final_data = final_data.drop(columns=['num_of_tracks_x', 'num_of_tracks_y'])

# Display the cleaned data
final_data.head()



In [ ]:
album_counts.columns

In [ ]:
# Rename columns in album_counts to match the ones in final_data for consistency
album_counts = album_counts.rename(columns={'album': 'Album Name', 'artist': 'Artist Name(s)'})

# Merge album_art into final_data based on 'Album Name' and 'Artist Name(s)'
final_data = pd.merge(final_data, album_counts[['Album Name', 'Artist Name(s)', 'album_art']], 
                      on=['Album Name', 'Artist Name(s)'], how='left')

# Display the final data with album art
final_data.head()


In [ ]:
final_data.columns

In [ ]:
num_tracks_best

In [ ]:
# Count the number of liked songs for each album
liked_counts = liked.groupby(['Album Name', 'Artist Name(s)'])['Track ID'].count().reset_index()

# Rename the 'Track ID' column to 'liked_count' to clarify what it represents
liked_counts = liked_counts.rename(columns={'Track ID': 'liked_count'})

# Merge the liked_count into final_data
final_data = pd.merge(final_data, liked_counts, on=['Album Name', 'Artist Name(s)'], how='left')

# Display the updated final_data with the count of liked songs per album
final_data.head()

In [ ]:
scrobbles.columns

In [ ]:
album_counts.columns

In [ ]:
final_data.columns

In [ ]:
# Merge 'scrobb_num' from album_counts into final_data
final_data = pd.merge(final_data, album_counts[['Album Name', 'Artist Name(s)', 'scrobb_num']], 
                      on=['Album Name', 'Artist Name(s)'], how='left')

# Rename 'scrobb_num' to 'faved'
final_data = final_data.rename(columns={'scrobb_num': 'scrobbles'})

# Display the updated final_data with the 'faved' column
final_data.head()


In [ ]:
final_data.to_excel('Data/2024_cleaned_preview.xlsx', index = False)